In [1]:
# Import Libraries

import optuna
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder, StandardScaler
pd.set_option('display.max_columns', None)

In [2]:
# Load in the datase

df = pd.read_csv("train.csv")

In [3]:
# Check to see what the dataset look like

df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
# check the info() of the dataset

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [5]:
# extracting the categorical and numeric variable names
num_cols = []
cat_cols = []
for col in df.columns:
    if df[col].dtypes == 'object':
        cat_cols.append(col)
    else:
        num_cols.append(col)

In [6]:
# Creating a numeric and catrgorical dataset

num_df = df[num_cols]
cat_df = df[cat_cols]

# Exploring the numeric dataframe

In [7]:
# Cheching to see the null values in the numric dataset

num_df.isnull().sum()

Id                 0
MSSubClass         0
LotFrontage      259
LotArea            0
OverallQual        0
OverallCond        0
YearBuilt          0
YearRemodAdd       0
MasVnrArea         8
BsmtFinSF1         0
BsmtFinSF2         0
BsmtUnfSF          0
TotalBsmtSF        0
1stFlrSF           0
2ndFlrSF           0
LowQualFinSF       0
GrLivArea          0
BsmtFullBath       0
BsmtHalfBath       0
FullBath           0
HalfBath           0
BedroomAbvGr       0
KitchenAbvGr       0
TotRmsAbvGrd       0
Fireplaces         0
GarageYrBlt       81
GarageCars         0
GarageArea         0
WoodDeckSF         0
OpenPorchSF        0
EnclosedPorch      0
3SsnPorch          0
ScreenPorch        0
PoolArea           0
MiscVal            0
MoSold             0
YrSold             0
SalePrice          0
dtype: int64

In [8]:
cat_df.isnull().sum()

MSZoning            0
Street              0
Alley            1369
LotShape            0
LandContour         0
Utilities           0
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
RoofStyle           0
RoofMatl            0
Exterior1st         0
Exterior2nd         0
MasVnrType          8
ExterQual           0
ExterCond           0
Foundation          0
BsmtQual           37
BsmtCond           37
BsmtExposure       38
BsmtFinType1       37
BsmtFinType2       38
Heating             0
HeatingQC           0
CentralAir          0
Electrical          1
KitchenQual         0
Functional          0
FireplaceQu       690
GarageType         81
GarageFinish       81
GarageQual         81
GarageCond         81
PavedDrive          0
PoolQC           1453
Fence            1179
MiscFeature      1406
SaleType            0
SaleCondition       0
dtype: int64

# Num_df

## RandomForest

In [9]:
num_df.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,0,150,856,856,854,0,1710,1,0,2,1,3,1,8,0,2003.0,2,548,0,61,0,0,0,0,0,2,2008,208500
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,0,284,1262,1262,0,0,1262,0,1,2,0,3,1,6,1,1976.0,2,460,298,0,0,0,0,0,0,5,2007,181500
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,0,434,920,920,866,0,1786,1,0,2,1,3,1,6,1,2001.0,2,608,0,42,0,0,0,0,0,9,2008,223500
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,0,540,756,961,756,0,1717,1,0,1,0,3,1,7,1,1998.0,3,642,0,35,272,0,0,0,0,2,2006,140000
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,0,490,1145,1145,1053,0,2198,1,0,2,1,4,1,9,1,2000.0,3,836,192,84,0,0,0,0,0,12,2008,250000


In [10]:
# Splitting the numerical dataset into X nad y

num_x = num_df.drop(columns=['SalePrice'], axis=1)
num_y = num_df.SalePrice

In [11]:
# Creating training and testing set

x_train,x_test,y_train,y_test = train_test_split(num_x,num_y,train_size=.80,random_state=42)

In [12]:
# Making a pipeline to catter for imputing missing values, scale the dataset and a classifier
pipe = make_pipeline(SimpleImputer(),StandardScaler(),RandomForestRegressor())

In [13]:
# fit the pipeline

pipe.fit(x_train, y_train)

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler()),
                ('randomforestregressor', RandomForestRegressor())])

In [14]:
# Predict with the pipeline

y_pred = pipe.predict(x_test)

In [15]:
# Get the mean absolute error

mean_absolute_error(y_test, y_pred)

17865.684554794523

In [16]:
pipe = make_pipeline(SimpleImputer(),StandardScaler(),RandomForestRegressor(random_state=46))

# fit the pipeline

pipe.fit(x_train, y_train)

# Predict with the pipeline

y_pred = pipe.predict(x_test)

# Get the mean absolute error

mean_absolute_error(y_test, y_pred)

17839.04825342466

## XGBOOSTING

In [18]:
pipex = make_pipeline(SimpleImputer(strategy="constant", fill_value=0),StandardScaler()
                      ,xgb.XGBRegressor())
pipex.fit(x_train, y_train)
y_predx = pipex.predict(x_test)
mean_absolute_error(y_test, y_predx)

18428.790012307363

## LogisticRegression

In [19]:
pipel = make_pipeline(SimpleImputer(),StandardScaler(),LinearRegression())
pipel.fit(x_train, y_train)
y_predl = pipel.predict(x_test)
mean_absolute_error(y_test, y_predl)

23012.022469997188

In [20]:
pd.Series(pipe['randomforestregressor'].feature_importances_,
          index=num_x.columns).sort_values(ascending=False).head(20)

OverallQual     0.570170
GrLivArea       0.127725
TotalBsmtSF     0.037306
BsmtFinSF1      0.032257
1stFlrSF        0.028653
2ndFlrSF        0.026111
LotArea         0.021653
GarageCars      0.019007
GarageArea      0.018594
YearBuilt       0.016212
LotFrontage     0.011381
YearRemodAdd    0.009801
GarageYrBlt     0.008242
OpenPorchSF     0.007452
TotRmsAbvGrd    0.006714
Id              0.005680
OverallCond     0.005607
BsmtUnfSF       0.005544
WoodDeckSF      0.005328
MasVnrArea      0.005127
dtype: float64

In [21]:
pd.Series(pipe['randomforestregressor'].feature_importances_,
          index=num_x.columns).sort_values(ascending=False).head(20).index

Index(['OverallQual', 'GrLivArea', 'TotalBsmtSF', 'BsmtFinSF1', '1stFlrSF',
       '2ndFlrSF', 'LotArea', 'GarageCars', 'GarageArea', 'YearBuilt',
       'LotFrontage', 'YearRemodAdd', 'GarageYrBlt', 'OpenPorchSF',
       'TotRmsAbvGrd', 'Id', 'OverallCond', 'BsmtUnfSF', 'WoodDeckSF',
       'MasVnrArea'],
      dtype='object')

In [22]:
# Load in the datase

df = pd.read_csv("train.csv")

In [23]:
best_20 = ['OverallQual', 'GrLivArea', 'TotalBsmtSF', 'BsmtFinSF1', '1stFlrSF',
       '2ndFlrSF', 'LotArea', 'GarageCars', 'GarageArea', 'YearBuilt',
       'LotFrontage', 'YearRemodAdd', 'GarageYrBlt', 'OpenPorchSF',
       'TotRmsAbvGrd', 'Id', 'OverallCond', 'BsmtUnfSF', 'WoodDeckSF',
       'MasVnrArea','SalePrice']

In [24]:
df_1 = df[best_20]

In [25]:
df_1.head()

,OverallQual,GrLivArea,TotalBsmtSF,BsmtFinSF1,1stFlrSF,2ndFlrSF,LotArea,GarageCars,GarageArea,YearBuilt,LotFrontage,YearRemodAdd,GarageYrBlt,OpenPorchSF,TotRmsAbvGrd,Id,OverallCond,BsmtUnfSF,WoodDeckSF,MasVnrArea,SalePrice
0,7,1710,856,706,856,854,8450,2,548,2003,65.0,2003,2003.0,61,8,1,5,150,0,196.0,208500
1,6,1262,1262,978,1262,0,9600,2,460,1976,80.0,1976,1976.0,0,6,2,8,284,298,0.0,181500
2,7,1786,920,486,920,866,11250,2,608,2001,68.0,2002,2001.0,42,6,3,5,434,0,162.0,223500
3,7,1717,756,216,961,756,9550,3,642,1915,60.0,1970,1998.0,35,7,4,5,540,0,0.0,140000
4,8,2198,1145,655,1145,1053,14260,3,836,2000,84.0,2000,2000.0,84,9,5,5,490,192,350.0,250000


In [26]:
x= df_1.drop(columns=['SalePrice'], axis=1)
y = df_1.SalePrice

In [27]:
# Creating training and testing set

x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=.80,random_state=42)

In [28]:
pipe = make_pipeline(SimpleImputer(),StandardScaler(),RandomForestRegressor(random_state=46))

# fit the pipeline

pipe.fit(x_train, y_train)

# Predict with the pipeline

y_pred = pipe.predict(x_test)

# Get the mean absolute error

mean_absolute_error(y_test, y_pred)

17977.973287671233

In [29]:
pipe = make_pipeline(SimpleImputer(),StandardScaler(),xgb.XGBRegressor())

# fit the pipeline

pipe.fit(x_train, y_train)

# Predict with the pipeline

y_pred = pipe.predict(x_test)

# Get the mean absolute error

mean_absolute_error(y_test, y_pred)

18192.446396083047

# Cat_df

In [30]:
xc = pd.get_dummies(cat_df)
yc = num_df.SalePrice

In [31]:
xc_train,xc_test,yc_train,yc_test = train_test_split(xc,yc,train_size=.80,random_state=42)

In [32]:
# Random Forest on categorical variables alone
pipec = make_pipeline(SimpleImputer(strategy='median'),
                      RandomForestRegressor())

In [33]:
pipec.fit(xc_train, yc_train)

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                ('randomforestregressor', RandomForestRegressor())])

In [34]:
y_predc = pipec.predict(xc_test)

In [35]:
mean_absolute_error(yc_test, y_predc)

24467.82851353555

In [36]:
# XGBoost on categorical variables alone

pipec = make_pipeline(SimpleImputer(strategy="constant", fill_value=0)
                      ,xgb.XGBRegressor())
pipec.fit(xc_train, yc_train)
y_predc = pipec.predict(xc_test)
mean_absolute_error(yc_test, y_predc)

24937.01907641267

In [37]:
pd.Series(pipec.named_steps.xgbregressor.feature_importances_,
          index =xc.columns).sort_values(ascending=False)

ExterQual_TA           0.294260
BsmtQual_Ex            0.141668
ExterQual_Fa           0.093701
KitchenQual_Ex         0.041081
BsmtQual_Gd            0.021903
                         ...   
BsmtCond_Po            0.000000
Electrical_Mix         0.000000
RoofMatl_Roll          0.000000
Exterior2nd_ImStucc    0.000000
Heating_GasA           0.000000
Length: 252, dtype: float32

In [38]:
new = pd.concat([num_df, xc],axis=1)

In [39]:
new.shape

(1460, 290)

In [40]:
new.isnull().sum()

Id                         0
MSSubClass                 0
LotFrontage              259
LotArea                    0
OverallQual                0
                        ... 
SaleCondition_AdjLand      0
SaleCondition_Alloca       0
SaleCondition_Family       0
SaleCondition_Normal       0
SaleCondition_Partial      0
Length: 290, dtype: int64

In [41]:
X = new.drop(columns=['SalePrice'], axis=1)
y = new.SalePrice

X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=.80,random_state=42)


# XGBoost on categorical variables alone

pipe = make_pipeline(SimpleImputer(strategy="constant", fill_value=0)
                      ,xgb.XGBRegressor())
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
mean_absolute_error(y_test, y_pred)

17502.048199379282

In [42]:
# X = new.drop(columns=['SalePrice'], axis=1)
# y = new.SalePrice

# X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=.80,random_state=42)


# XRandomForest on categorical variables alone

piperf = make_pipeline(SimpleImputer(strategy="constant", fill_value=0)
                          ,RandomForestRegressor(random_state=12))
piperf.fit(X_train, y_train)
y_predrf = piperf.predict(X_test)
mean_absolute_error(y_test, y_predrf)

17376.821712328765

In [43]:
import optuna

def objective(trial):
    # Define X and y
    X = new.drop(columns=['SalePrice'], axis=1)
    y = new.SalePrice
    # Define training and testing set
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=.80,random_state=42)
    
    # Define parameters
    
    learning_rate = trial.suggest_float("learning_rate", 0.1,2)
    max_depth = trial.suggest_int("max_depth", 3,5)
    n_estimators = trial.suggest_int("n_estimators", 100,200)
    max_leaves = trial.suggest_int("max_leaves", 3,5)
    
    pipe = make_pipeline(SimpleImputer(),
                        xgb.XGBRegressor(learning_rate=learning_rate,
                                        max_depth=max_depth,
                                        n_estimators=n_estimators,
                                        max_leaves=max_leaves))
    
    pipe.fit(X_train, y_train)
    
    y_pred = pipe.predict(X_test)
    
    return mean_absolute_error(y_test, y_pred)


study = optuna.create_study(direction="minimize")

study.optimize(objective, n_trials=100)


print(study.best_params)
print(study.best_value)

[I 2023-06-03 17:39:25,830] A new study created in memory with name: no-name-8f4c7a79-0ea8-4a7c-a64c-ccc94034c822
[I 2023-06-03 17:39:26,562] Trial 0 finished with value: 21325.436188998287 and parameters: {'learning_rate': 0.8505556894600437, 'max_depth': 3, 'n_estimators': 192, 'max_leaves': 4}. Best is trial 0 with value: 21325.436188998287.
[I 2023-06-03 17:39:27,169] Trial 1 finished with value: 57632.58056640625 and parameters: {'learning_rate': 1.9688349710083606, 'max_depth': 3, 'n_estimators': 163, 'max_leaves': 3}. Best is trial 0 with value: 21325.436188998287.
[I 2023-06-03 17:39:27,914] Trial 2 finished with value: 35698.61690322667 and parameters: {'learning_rate': 1.4558752548619331, 'max_depth': 4, 'n_estimators': 158, 'max_leaves': 4}. Best is trial 0 with value: 21325.436188998287.
[I 2023-06-03 17:39:28,833] Trial 3 finished with value: 65728.447947881 and parameters: {'learning_rate': 1.9284821269114776, 'max_depth': 5, 'n_estimators': 158, 'max_leaves': 4}. Best is

[I 2023-06-03 17:39:47,529] Trial 33 finished with value: 15970.992334653254 and parameters: {'learning_rate': 0.19936628918765328, 'max_depth': 4, 'n_estimators': 113, 'max_leaves': 3}. Best is trial 33 with value: 15970.992334653254.
[I 2023-06-03 17:39:48,295] Trial 34 finished with value: 19104.961004387842 and parameters: {'learning_rate': 0.37396193047424187, 'max_depth': 4, 'n_estimators': 136, 'max_leaves': 3}. Best is trial 33 with value: 15970.992334653254.
[I 2023-06-03 17:39:48,812] Trial 35 finished with value: 16647.384631849316 and parameters: {'learning_rate': 0.22728552992232742, 'max_depth': 3, 'n_estimators': 112, 'max_leaves': 3}. Best is trial 33 with value: 15970.992334653254.
[I 2023-06-03 17:39:49,797] Trial 36 finished with value: 18871.22158604452 and parameters: {'learning_rate': 0.47285559079760886, 'max_depth': 4, 'n_estimators': 169, 'max_leaves': 4}. Best is trial 33 with value: 15970.992334653254.
[I 2023-06-03 17:39:50,471] Trial 37 finished with value:

[I 2023-06-03 17:40:09,259] Trial 67 finished with value: 16331.786159567637 and parameters: {'learning_rate': 0.16244472697988463, 'max_depth': 4, 'n_estimators': 125, 'max_leaves': 3}. Best is trial 66 with value: 15887.934142230308.
[I 2023-06-03 17:40:09,900] Trial 68 finished with value: 17214.40127354452 and parameters: {'learning_rate': 0.2617784034000865, 'max_depth': 4, 'n_estimators': 133, 'max_leaves': 3}. Best is trial 66 with value: 15887.934142230308.
[I 2023-06-03 17:40:10,499] Trial 69 finished with value: 18224.772246896406 and parameters: {'learning_rate': 0.3586664954865806, 'max_depth': 4, 'n_estimators': 125, 'max_leaves': 3}. Best is trial 66 with value: 15887.934142230308.
[I 2023-06-03 17:40:11,144] Trial 70 finished with value: 16294.457900791953 and parameters: {'learning_rate': 0.15553012030527152, 'max_depth': 4, 'n_estimators': 136, 'max_leaves': 4}. Best is trial 66 with value: 15887.934142230308.
[I 2023-06-03 17:40:11,824] Trial 71 finished with value: 1

{'learning_rate': 0.19793614054048989, 'max_depth': 4, 'n_estimators': 155, 'max_leaves': 5}
15870.381728916953


In [44]:
def objective(trial):
    # Define X and y
    X = new.drop(columns=['SalePrice'], axis=1)
    y = new.SalePrice
    # Define training and testing set
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=.80,random_state=42)
    
    # Define parameters
    
    max_depth = trial.suggest_int("max_depth", 3,5)
    n_estimators = trial.suggest_int("n_estimators", 100,200)
#    max_leaf_nodes = trial.suggest_int("max_leaf_nodes", 3,5)
    min_samples_split = trial.suggest_int("min_samples_split", 2,6)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1,5)
#     max_features = trial.suggest_categorical("max_features", ["sqrt", "log2"])
    
    pipe = make_pipeline(SimpleImputer(),
                        RandomForestRegressor(max_depth=max_depth,
                                        n_estimators=n_estimators,
                                              min_samples_leaf=min_samples_leaf,
                                              min_samples_split=min_samples_split))
                                            #max_leaf_nodes=max_leaf_nodes,
                                        #max_features=max_features))
    
    pipe.fit(X_train, y_train)
    
    y_pred = pipe.predict(X_test)
    
    return mean_absolute_error(y_test, y_pred)


study = optuna.create_study(direction="minimize")

study.optimize(objective, n_trials=100)


print(study.best_params)
print(study.best_value)

[I 2023-06-03 17:40:30,704] A new study created in memory with name: no-name-d958f8c5-a63c-4957-9525-d2f9d7a6318a
[I 2023-06-03 17:40:32,284] Trial 0 finished with value: 21198.593660474595 and parameters: {'max_depth': 5, 'n_estimators': 156, 'min_samples_split': 6, 'min_samples_leaf': 1}. Best is trial 0 with value: 21198.593660474595.
[I 2023-06-03 17:40:33,286] Trial 1 finished with value: 25740.259107076257 and parameters: {'max_depth': 3, 'n_estimators': 157, 'min_samples_split': 2, 'min_samples_leaf': 2}. Best is trial 0 with value: 21198.593660474595.
[I 2023-06-03 17:40:34,266] Trial 2 finished with value: 20644.46986615603 and parameters: {'max_depth': 5, 'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 1}. Best is trial 2 with value: 20644.46986615603.
[I 2023-06-03 17:40:35,247] Trial 3 finished with value: 26027.114078669147 and parameters: {'max_depth': 3, 'n_estimators': 153, 'min_samples_split': 3, 'min_samples_leaf': 2}. Best is trial 2 with value: 2064

[I 2023-06-03 17:41:19,953] Trial 34 finished with value: 20917.689250865868 and parameters: {'max_depth': 5, 'n_estimators': 107, 'min_samples_split': 5, 'min_samples_leaf': 1}. Best is trial 33 with value: 20455.222997832676.
[I 2023-06-03 17:41:20,790] Trial 35 finished with value: 26056.514876401194 and parameters: {'max_depth': 3, 'n_estimators': 125, 'min_samples_split': 6, 'min_samples_leaf': 1}. Best is trial 33 with value: 20455.222997832676.
[I 2023-06-03 17:41:22,014] Trial 36 finished with value: 21000.665669819136 and parameters: {'max_depth': 5, 'n_estimators': 121, 'min_samples_split': 6, 'min_samples_leaf': 2}. Best is trial 33 with value: 20455.222997832676.
[I 2023-06-03 17:41:23,077] Trial 37 finished with value: 20915.453890956735 and parameters: {'max_depth': 5, 'n_estimators': 107, 'min_samples_split': 5, 'min_samples_leaf': 2}. Best is trial 33 with value: 20455.222997832676.
[I 2023-06-03 17:41:23,986] Trial 38 finished with value: 22620.09835903978 and paramete

[I 2023-06-03 17:42:12,686] Trial 69 finished with value: 20891.49594751014 and parameters: {'max_depth': 5, 'n_estimators': 176, 'min_samples_split': 4, 'min_samples_leaf': 1}. Best is trial 42 with value: 20381.740310504483.
[I 2023-06-03 17:42:13,983] Trial 70 finished with value: 25918.445337482837 and parameters: {'max_depth': 3, 'n_estimators': 196, 'min_samples_split': 2, 'min_samples_leaf': 2}. Best is trial 42 with value: 20381.740310504483.
[I 2023-06-03 17:42:15,855] Trial 71 finished with value: 20528.582061157613 and parameters: {'max_depth': 5, 'n_estimators': 188, 'min_samples_split': 4, 'min_samples_leaf': 3}. Best is trial 42 with value: 20381.740310504483.
[I 2023-06-03 17:42:17,624] Trial 72 finished with value: 20678.201445596656 and parameters: {'max_depth': 5, 'n_estimators': 178, 'min_samples_split': 4, 'min_samples_leaf': 4}. Best is trial 42 with value: 20381.740310504483.
[I 2023-06-03 17:42:19,047] Trial 73 finished with value: 20825.795162880244 and paramete

{'max_depth': 5, 'n_estimators': 128, 'min_samples_split': 4, 'min_samples_leaf': 2}
20381.740310504483


In [45]:
pipe = make_pipeline(SimpleImputer(),
                        xgb.XGBRegressor(learning_rate=0.1650,
                                        max_depth=5,
                                        n_estimators=300,
                                        max_leaves=3))
    
pipe.fit(X_train, y_train)
    
y_pred = pipe.predict(X_test)
    
mean_absolute_error(y_test, y_pred)

15386.986287992295

In [46]:
pipe = make_pipeline(SimpleImputer(),
                     xgb.XGBRegressor(learning_rate= 0.1650,
                                      max_depth= 5,n_estimators= 300,
                                     max_leaves=3))

pipe.fit(X_train, y_train)
    
y_pred = pipe.predict(X_test)
    
mean_absolute_error(y_test, y_pred)

15386.986287992295